# 📂 IoT Sensor Anomaly Detection
This notebook demonstrates anomaly detection on IoT sensor data using Z-Score, Isolation Forest, and Rolling Mean methods.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest


In [ ]:

# Load dataset
df = pd.read_csv("../data/sensor.csv", parse_dates=["date"])
df.set_index("date", inplace=True)

# Preview
df.head()


## 1️⃣ Z-Score Anomaly Detection

In [ ]:

df["z_score"] = (df["temperature"] - df["temperature"].mean()) / df["temperature"].std()
df["z_anomaly"] = df["z_score"].apply(lambda x: 1 if np.abs(x) > 2.5 else 0)

# Plot
plt.figure(figsize=(14,5))
plt.plot(df.index, df["temperature"], marker="o", label="Temperature")
plt.scatter(df.index[df["z_anomaly"]==1], df["temperature"][df["z_anomaly"]==1],
            color="red", label="Z-Score Anomaly", s=80)
plt.title("Z-Score Anomaly Detection")
plt.xlabel("Date")
plt.ylabel("Temperature (°C)")
plt.legend()
plt.show()


## 2️⃣ Isolation Forest Anomaly Detection

In [ ]:

model = IsolationForest(contamination=0.05, random_state=42)
df["iforest_anomaly"] = model.fit_predict(df[["temperature"]])
df["iforest_anomaly"] = df["iforest_anomaly"].apply(lambda x: 1 if x == -1 else 0)

# Plot
plt.figure(figsize=(14,5))
plt.plot(df.index, df["temperature"], marker="o", label="Temperature")
plt.scatter(df.index[df["iforest_anomaly"]==1], df["temperature"][df["iforest_anomaly"]==1],
            color="blue", label="Isolation Forest Anomaly", s=80)
plt.title("Isolation Forest Anomaly Detection")
plt.xlabel("Date")
plt.ylabel("Temperature (°C)")
plt.legend()
plt.show()


## 3️⃣ Rolling Mean / Moving Average Anomaly Detection

In [ ]:

window = 7  # 7-day rolling window
df["rolling_mean"] = df["temperature"].rolling(window=window).mean()
df["rolling_std"] = df["temperature"].rolling(window=window).std()
df["rolling_anomaly"] = ((df["temperature"] - df["rolling_mean"]).abs() > 2.5 * df["rolling_std"]).astype(int)

# Plot
plt.figure(figsize=(14,5))
plt.plot(df.index, df["temperature"], marker="o", label="Temperature")
plt.plot(df.index, df["rolling_mean"], linestyle="--", color="green", label="Rolling Mean")
plt.scatter(df.index[df["rolling_anomaly"]==1], df["temperature"][df["rolling_anomaly"]==1],
            color="orange", label="Rolling Mean Anomaly", s=80)
plt.title("Rolling Mean Anomaly Detection")
plt.xlabel("Date")
plt.ylabel("Temperature (°C)")
plt.legend()
plt.show()


## 4️⃣ Save Anomalies to CSV

In [ ]:

df.to_csv("../data/sensor_anomalies_detected.csv")
print("Anomaly detection complete. Results saved to 'sensor_anomalies_detected.csv'")
